# Tổng quan đề tài

Cơ sở dữ liệu bóng đá châu Âu có hơn 25.000 trận đấu và hơn 10.000 cầu thủ cho các mùa bóng đá chuyên nghiệp châu Âu từ 2008 đến 2016. Mục tiêu là xem qua cơ sở dữ liệu này và thực hiện phân tích, bao gồm một số bước khám phá dữ liệu, thống kê cơ bản và sau đó hình dung kết quả. Đề tài này thực hành viết lệnh SQL để kéo dữ liệu về và trích xuất nó.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Mô tả dữ liệu

Cơ sở dữ liệu bóng đá châu Âu này có hơn 25.000 trận đấu và hơn 10.000 cầu thủ cho các mùa bóng đá chuyên nghiệp châu Âu từ 2008 đến 2016, 11 quốc gia châu Âu với chức vô địch dẫn đầu. Các thuộc tính của Người chơi và Đội có nguồn gốc từ loạt trò chơi video FIFA của EA Sports, bao gồm các bản cập nhật hàng tuần.

# Gọi thư viện Python

In [ ]:
#Import libraries
import numpy as np
import pandas as pd
import sqlite3
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)

## Kết nối tới cơ sở dữ liệu và truy vấn thông tin của tất cả các bảng

Đọc https://docs.python.org/2/library/sqlite3.html

In [ ]:
# Kết nối với cơ sở dữ liệu
conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/DSP302/ASM1/database.sqlite')

# Lấy thông tin của tất cả bảng
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


## Liệt kê các quốc gia có trong dữ liệu bảng "Country"

In [ ]:
# Liệt kê các quốc gia có trong dữ liệu bảng "Country"
q2 = pd.read_sql('''SELECT name AS Country 
                    FROM Country''', conn)
q2

,Country
0,Belgium
1,England
2,France
3,Germany
4,Italy
5,Netherlands
6,Poland
7,Portugal
8,Scotland
9,Spain


## Liệt kê các giải đấu trong bảng "League"

In [ ]:
# Liệt kê các giải đấu trong bảng "League"
q3 = pd.read_sql('''SELECT name AS League 
                    FROM League''', conn)
q3

,League
0,Belgium Jupiler League
1,England Premier League
2,France Ligue 1
3,Germany 1. Bundesliga
4,Italy Serie A
5,Netherlands Eredivisie
6,Poland Ekstraklasa
7,Portugal Liga ZON Sagres
8,Scotland Premier League
9,Spain LIGA BBVA


## Liệt kê các giải đấu và thông tin quốc gia của từng league

Bảng League có quan hệ với bảng Country qua khóa country_id.

In [ ]:
# Liệt kê các giải đấu và thông tin quốc gia của từng league
# Trộn 2 bảng, Country và League rồi lấy tên League, Country với điều kiện c.id=l.country_id
q4 = pd.read_sql('''SELECT L.name AS League, C.name AS Country 
                    FROM Country AS c, League AS l 
                    WHERE c.id=l.country_id''', conn)
q4

,League,Country
0,Belgium Jupiler League,Belgium
1,England Premier League,England
2,France Ligue 1,France
3,Germany 1. Bundesliga,Germany
4,Italy Serie A,Italy
5,Netherlands Eredivisie,Netherlands
6,Poland Ekstraklasa,Poland
7,Portugal Liga ZON Sagres,Portugal
8,Scotland Premier League,Scotland
9,Spain LIGA BBVA,Spain


## Liệt kê các trận đấu trong bảng  “Match”
- Khi một đội là chủ nhà, đội đó là "home team". Đội còn lại là "away team"
- Trong bảng Match, mỗi dòng dư liệu là một trận đấu của một đội chủ nhà và đội khách, tương ứng với kết quả ghi bàn đội nhà (home goal) và đội khách (away goal).

In [ ]:
# Liệt kê các trận đấu trong bảng “Match”
q5 = pd.read_sql('''SELECT * FROM Match''', conn)
q5

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,...,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,...,195215.0,462944.0,

## Liệt kê các trận đấu và thông tin về leage và country tương ứng

Bảng Match có quan hệ với :
 - Bảng Country thông qua khóa country_id
 - Bảng League thông qua khóa League_id
 
Sử dụng câu lệnh join để kết nối dư liệu từ 3 bảng trên.


In [ ]:
# Liệt kê các trận đấu và thông tin về leage và country tương ứng
# League và Country được liệt kê ở cột cuối bảng 
q6 = pd.read_sql('''SELECT m.*, l.name AS League, c.name AS Country 
                    FROM Match AS m 
                    JOIN Country AS c ON m.country_id=c.id 
                    JOIN League AS l ON m.league_id=l.id''', conn)
q6

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,League,Country
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20,Belgium Jupiler League,Belgium
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60,Belgium Jupiler League,Belgium
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75,Belgium Jupiler League,Belgium
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50,Belgium Jupiler League,Belgium
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67,Belgium Jupiler League,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.

## Liệt kê số lượng trận đấu của mỗi giải đấu bao gồm cả tên giải đấu, sắp xếp theo thứ tự giảm dần số trận đấu

In [ ]:
# Liệt kê số lượng trận đấu của mỗi giải đấu bao gồm cả tên giải đấu, sắp xếp theo thứ tự giảm dần số trận đấu
q7 = pd.read_sql('''SELECT l.name AS League, count(*) AS Number_of_Match 
                    FROM League AS l, Match AS m 
                    WHERE m.league_id=l.id 
                    GROUP BY League 
                    ORDER BY Number_of_Match DESC''', conn)
q7

,League,Number_of_Match
0,England Premier League,3040
1,France Ligue 1,3040
2,Spain LIGA BBVA,3040
3,Italy Serie A,3017
4,Germany 1. Bundesliga,2448
5,Netherlands Eredivisie,2448
6,Portugal Liga ZON Sagres,2052
7,Poland Ekstraklasa,1920
8,Scotland Premier League,1824
9,Belgium Jupiler League,1728


## Liệt kê tổng số bàn thắng của đội nhà và đội khách trong mỗi giải đấu
- Sử dụng câu lệnh Group By

In [ ]:
# Liệt kê tổng số bàn thắng của đội nhà và đội khách trong mỗi giải đấu
q8 = pd.read_sql('''SELECT l.name AS League, 
                           sum(home_team_goal) AS Home_Goal, 
                           sum(away_team_goal) As Away_Goal
                    FROM League AS l, Match as m 
                    WHERE m.league_id=l.id 
                    GROUP BY League''', conn)
q8

,League,Home_Goal,Away_Goal
0,Belgium Jupiler League,2781,2060
1,England Premier League,4715,3525
2,France Ligue 1,4265,3162
3,Germany 1. Bundesliga,3982,3121
4,Italy Serie A,4528,3367
5,Netherlands Eredivisie,4357,3185
6,Poland Ekstraklasa,2678,1978
7,Portugal Liga ZON Sagres,2890,2311
8,Scotland Premier League,2607,2197
9,Spain LIGA BBVA,4959,3453


## Liệt kê thông tin các đội từ bảng “Team”

In [ ]:
# Liệt kê thông tin các đội từ bảng “Team”
q9 = pd.read_sql("select * from Team", conn)
q9

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


## Liệt kê 20 đội với số bàn thắng sân nhà cao nhất
- Sử dung câu lênh GROUP BY, LIMIT

In [ ]:
# Liệt kê 20 đội với số bàn thắng sân nhà cao nhất
q10 = pd.read_sql('''SELECT t.team_long_name AS Team, sum(home_team_goal) AS Goal 
                     FROM Team AS t, Match AS m 
                     where t.team_api_id=m.home_team_api_id 
                     GROUP BY Team
                     ORDER BY Goal DESC 
                     LIMIT 20''', conn)
q10

,Team,Goal
0,Real Madrid CF,505
1,FC Barcelona,495
2,Celtic,389
3,FC Bayern Munich,382
4,PSV,370
5,Manchester City,365
6,Ajax,360
7,FC Basel,344
8,Manchester United,338
9,Chelsea,333


## Liệt kê 20 đội có số bàn thắng trên sân khách cao nhất
- Sử dụng câu lênh GROUP BY, LIMIT

In [ ]:
# Liệt kê 20 đội có số bàn thắng trên sân khách cao nhất
q11 = pd.read_sql('''SELECT t.team_long_name as Team, sum(away_team_goal) as Goal 
                     FROM Team as t, Match as m 
                     WHERE t.team_api_id=m.home_team_api_id 
                     GROUP BY Team
                     ORDER BY Goal DESC 
                     LIMIT 20''', conn)
q11

,Team,Goal
0,Kilmarnock,238
1,SV Werder Bremen,203
2,SC Heerenveen,202
3,Motherwell,201
4,ADO Den Haag,200
5,Aston Villa,198
6,Hannover 96,195
7,VfB Stuttgart,195
8,Dundee United,194
9,Willem II,192


## Liệt kê tên các đội bóng và tổng số bàn thắng mỗi đội, sắp xếp  theo số lượng giảm dần

- Tổng số bàn thắng của một đội bao gồm cả số bàn thắng ở sân nhà và sân khách.
- Sử dụng câu lệnh Union  

In [ ]:
# Liệt kê tên các đội bóng và tổng số bàn thắng mỗi đội, sắp xếp theo số lượng giảm dần
q12 = pd.read_sql('''SELECT t.team_long_name AS Team, sum(home_team_goal+away_team_goal) AS Goal 
                     FROM Team AS t, Match AS m 
                     WHERE t.team_api_id=m.home_team_api_id 
                     GROUP BY Team
                     ORDER BY Goal DESC''', conn)
q12

,Team,Goal
0,Real Madrid CF,652
1,FC Barcelona,596
2,PSV,496
3,Manchester City,490
4,BSC Young Boys,485
...,...,...
291,Uniao da Madeira,36
292,Angers SCO,35
293,Trofense,35
294,Feirense,32


## Liệt kê tên các đội bóng (long name) và tổng số trận đấu đội đó tham gia, sắp xếp theo thứ tự giảm dần.

In [ ]:
# Liệt kê tên các đội bóng (long name) và tổng số trận đấu đội đó tham gia, sắp xếp theo thứ tự giảm dần.
q13 = pd.read_sql('''SELECT t.team_long_name AS Team, count(*) as Number_of_Match
                     FROM Team AS t, Match AS m 
                     WHERE t.team_api_id=m.home_team_api_id or t.team_api_id=m.away_team_api_id
                     GROUP BY Team
                     ORDER BY Number_of_Match DESC''', conn)
q13

,Team,Number_of_Match
0,AS Saint-Étienne,304
1,Aberdeen,304
2,Arsenal,304
3,Aston Villa,304
4,Athletic Club de Bilbao,304
...,...,...
291,Feirense,30
292,KAS Eupen,30
293,Portimonense,30
294,Termalica Bruk-Bet Nieciecza,30


## Liệt kê số trận thắng, thua và hòa của mỗi đội.

- Nếu một đội có số lượng bàn thắng trên sân nhà > số lượng bàn thắng trên sân khách của chính đội đó, thì đội đó sẽ "thắng" trong trận đấu này
- Nếu một đội có số lượng bàn thắng trên sân nhà < số lượng bàn thắng trên sân khách của chính đội đó, thì đội đó sẽ "thua" trong trận đấu này
- Nếu một đội có số lượng bàn thắng trên sân nhà = số lượng bàn thắng trên sân khách của chính đội đó, thì đội đó sẽ "hòa" trong trận đấu này
- Sử dụng câu lệnh "Case When"


In [ ]:
# Liệt kê số trận thắng, thua và hòa của mỗi đội.
q14 = pd.read_sql_query('''SELECT t.team_long_name AS Team,
                           COUNT(CASE WHEN m.home_team_goal > away_team_goal THEN 'Win' END) AS Won,
                           COUNT(CASE WHEN m.home_team_goal < away_team_goal THEN 'Lost' END) AS Lost,
                           COUNT(CASE WHEN m.home_team_goal = away_team_goal THEN 'Tie' END) AS Tie
                           FROM Match AS m
                           LEFT JOIN Team as t
                           ON m.home_team_api_id = t.team_api_id
                           GROUP BY Team''', conn)
q14

,Team,Won,Lost,Tie
0,1. FC Kaiserslautern,8,15,11
1,1. FC Köln,32,39,31
2,1. FC Nürnberg,30,34,21
3,1. FSV Mainz 05,55,35,29
4,AC Ajaccio,16,22,19
...,...,...,...,...
291,Xerez Club Deportivo,6,9,4
292,Zagłębie Lubin,35,28,27
293,Zawisza Bydgoszcz,13,12,5
294,Évian Thonon Gaillard FC,29,30,17


## Liệt kê 10 đội có số trận thắng nhiều nhất

In [ ]:
# Liệt kê 10 đội có số trận thắng nhiều nhất
q15 = pd.read_sql_query('''SELECT t.team_long_name AS Team,
                            COUNT(CASE WHEN m.home_team_goal > away_team_goal THEN 'Win' END) AS Won
                            FROM Match AS m
                            LEFT JOIN Team as t ON m.home_team_api_id = t.team_api_id
                            GROUP BY Team
                            ORDER BY Won DESC
                            LIMIT 10''', conn)
q15

,Team,Won
0,FC Barcelona,131
1,Real Madrid CF,129
2,Celtic,120
3,Manchester United,116
4,Manchester City,113
5,FC Bayern Munich,109
6,Juventus,105
7,PSV,105
8,Ajax,103
9,Atlético Madrid,103


## Liệt kê tỉ lệ phần trăm về số trận của mỗi giải trên tổng tất cả các trận

In [ ]:
# Liệt kê tỉ lệ phần trăm về số trận của mỗi giải trên tổng tất cả các trận
q16 = pd.read_sql('''SELECT l.name AS League, count(*)*100.0 / (select count(*) from Match) AS 'Percent_Match(%)'
                     FROM League AS l, Match AS m
                     WHERE l.id=m.league_id 
                     GROUP BY League''', conn)
q16

,League,Percent_Match(%)
0,Belgium Jupiler League,6.651526
1,England Premier League,11.701759
2,France Ligue 1,11.701759
3,Germany 1. Bundesliga,9.422995
4,Italy Serie A,11.613226
5,Netherlands Eredivisie,9.422995
6,Poland Ekstraklasa,7.390585
7,Portugal Liga ZON Sagres,7.898687
8,Scotland Premier League,7.021055
9,Spain LIGA BBVA,11.701759


## Liệt kê tỉ lệ phần trăm về số bàn ghi được của từng giải so với tổng số bàn của tất cả các trận.

In [ ]:
# Liệt kê tỉ lệ phần trăm về số bàn ghi được của từng giải so với tổng số bàn của tất cả các trận.
q18 = pd.read_sql('''SELECT name AS League,
                            SUM(home_team_goal+away_team_goal)*100.0/(select SUM(home_team_goal+away_team_goal) from Match) AS 'Percent_Goal(%)'
                      FROM Match AS m
                      LEFT JOIN League as l
                      ON m.league_id = l.id
                      GROUP BY League       
                      ''', conn)
q18

,League,Percent_Goal(%)
0,Belgium Jupiler League,6.887476
1,England Premier League,11.723363
2,France Ligue 1,10.566677
3,Germany 1. Bundesliga,10.105709
4,Italy Serie A,11.232518
5,Netherlands Eredivisie,10.730292
6,Poland Ekstraklasa,6.624269
7,Portugal Liga ZON Sagres,7.399661
8,Scotland Premier League,6.834834
9,Spain LIGA BBVA,11.968074


## Liệt kê tổng số bàn ghi được của từng giải đấu theo từng mùa giải.

In [ ]:
# Liệt kê tổng số bàn ghi được của từng giải đấu theo từng mùa giải.
q18 = pd.read_sql('''SELECT season AS Season,
                            name AS League,
                            SUM(home_team_goal+away_team_goal) AS Goal
                      FROM Match AS m
                      LEFT JOIN League as l
                      ON m.league_id = l.id
                      GROUP BY Season, League       
                      ''', conn)
q18

,Season,League,Goal
0,2008/2009,Belgium Jupiler League,855
1,2008/2009,England Premier League,942
2,2008/2009,France Ligue 1,858
3,2008/2009,Germany 1. Bundesliga,894
4,2008/2009,Italy Serie A,988
...,...,...,...
83,2015/2016,Poland Ekstraklasa,635
84,2015/2016,Portugal Liga ZON Sagres,831
85,2015/2016,Scotland Premier League,650
86,2015/2016,Spain LIGA BBVA,1043


## Liệt kê thông tin cầu thủ:

- Quy đổi cân nặng sang kilogram
- Quy đổi chiều cao sang mét
- Tính chỉ số bmi = ( (weight* 0.453592) / (height/100)^2)
- Tính tuổi của cầu thủ

In [ ]:
# Liệt kê thông tin cầu thủ gồm Tên, Chiều cao(Mét), Cân nặng(Kg), BMI, Tuổi
q19 = pd.read_sql('''select player_name as Name, 
                     height/100 as Height, 
                     weight*0.453592 as Weight,
                     (weight* 0.453592) / (height/100) as BMI,
                     current_date - birthday as Age
                     FROM Player''', conn)
q19

,Name,Height,Weight,BMI,Age
0,Aaron Appindangoye,1.8288,84.821704,46.381072,30
1,Aaron Cresswell,1.7018,66.224432,38.914345,33
2,Aaron Doran,1.7018,73.935496,43.445467,31
3,Aaron Galindo,1.8288,89.811216,49.109370,40
4,Aaron Hughes,1.8288,69.853168,38.196177,43
...,...,...,...,...,...
11055,Zoumana Camara,1.8288,76.203456,41.668556,43
11056,Zsolt Laczko,1.8288,79.832192,43.652773,36
11057,Zsolt Low,1.8034,69.853168,38.734151,43
11058,Zurab Khizanishvili,1.8542,78.017824,42.076272,41


## Liệt kê cầu thủ lớn tuổi nhất

In [ ]:
# Liệt kê cầu thủ lớn tuổi nhất
q20 = pd.read_sql('''select player_name as Name,
                     current_date - birthday as Age
                     FROM Player
                     ORDER BY Age DESC
                     LIMIT 1
                     ''', conn)
q20

,Name,Age
0,Alberto Fontana,55


## Liệt kê các cầu thủ có tổng số điểm đánh giá lớn hơn 80

In [ ]:
# Liệt kê các cầu thủ có tổng số điểm đánh giá lớn hơn 80
q22 = pd.read_sql('''select p.player_name as Name, pt.overall_rating	
                     FROM Player AS p, Player_Attributes AS pt
                     WHERE p.player_api_id=pt.player_api_id and pt.overall_rating>80
                     Group BY Name
                     ''', conn)
q22

,Name,overall_rating
0,Aaron Lennon,84
1,Aaron Ramsey,81
2,Abdulkader Keita,82
3,Adam Johnson,81
4,Adem Ljajic,81
...,...,...
832,Yuri Zhirkov,82
833,Zdenek Grygera,81
834,Ze Roberto,90
835,Zlatan Ibrahimovic,87
